## Default Settings

In [1]:
#pip install haversine   #Library 설치

In [2]:
import os.path as osp
import os
import numpy as np
import pandas as pd
import csv
import random
from haversine import haversine
import math
import datetime

path_dir ='./RawData'

file_list = os.listdir(path_dir)


# Make directory
if not osp.exists('./Result'):
    os.makedirs('./Result')

## 관련 함수 정의

In [3]:
def ChangedDate(x):
    x = str(int(x))
    if(len(x) == 5):
        x = "20" + x[3:5] + "-" + x[1:3] + "-0" + x[0:1]   
    elif(len(x) == 6):
        x = "20" + x[4:6] + "-" + x[2:4] + "-" + x[0:2]
    else:
        x = 'Error'
    
    return x

def ChangedTime(x):
    x = str(x)
    if(len(x) == 7):
        temp = int(x[0:1])
        temp += 9
        x = str(temp) + ':' + x[1:3]+ ':' + x[3:] + '0'
    elif(len(x) == 9):
        temp = int(x[0:2])
        temp += 9
        x = str(temp) + ':' + x[2:4]+ ':' + x[4:]
    else:
        x = 'Error'
    return x

## CSV 파일 전처리

In [4]:
for idx, f_name in enumerate(file_list):
    data = pd.read_csv(osp.join(path_dir, f_name) , sep=',', header=None)
    df = pd.DataFrame(data)

    df_interest = df.iloc[:-1,[1,3,5,9]] 
    df_interest.columns = ['Time', 'Latitude', 'Longitude', 'Date']

    id = f_name[:2]
    if id[-1] == '_':
        id = id[:-1]

    df_interest['ID'] = id
    df_interest['Flag'] = 1
    df_interest['Date'] = df_interest['Date'].apply(ChangedDate)
    df_interest['Time'] = df_interest['Time'].apply(ChangedTime)

    df_interest['Date-Time'] = df_interest[['Date','Time']].apply(lambda x: ' '.join(x), axis=1)
    df_final = df_interest[['ID', 'Date-Time', 'Flag', 'Latitude', 'Longitude']]
    
    df_final.to_csv(osp.join('Result', 'raw_data_{}.csv'.format(idx)))

## 거리 구하기

정확도와 오차<br>
GPS 수신기에서 위치를 계산하기 위해서는 현재의 시각, 위성의 위치, 신호의 지연량이 필요하다. 위치 계산 오차는 이 가운데 주로 위성의 위치와 신호 지연의 측정으로부터 발생한다.<br>

신호의 지연 시간은 GPS 위성으로부터 수신한 신호와 동일한 신호를 GPS 수신기에서 발생시켜 비교하여 얻는다. 이 비교 과정에서 발생하는 오차는 수신기의 수신 상태가 양호한 경우, 부호 길이의 1% 정도이므로 C/A 코드에서는 약 1~10 ㎱이다. 전파의 속도를 고려하면 1~3 미터 정도의 오차이다. 이는 신호 지연 측정 과정에서 발생하는 오차의 최소치이다. P(Y) 코드를 해독할 수 있는 경우, 부호 길이의 1%의 오차는 약 30 센티미터에 해당한다.<br>

이 밖에 다음과 같은 오차가 발생한다.<br>

전리층의 영향: ± 5 미터<br>
천체력 오차: ± 2.5 미터<br>
위성의 시계 오차: ± 2 미터<br>
전파 경로에 따른 오차: ± 1 미터<br>
대류권의 영향: ± 0.5 미터<br>
수치 오차: ± 1 미터 이하<br>

<br>
천체력 및 위성 시계 오차 : 이와 같은 오차는 전리층 오차와 달리 몇 날 또는 몇 주간에 걸쳐서 변화하기 때문에 보다 안정적인 편에 속한다. 그러므로 더 정확한 궤도 정보와 이력(almanac)을 별개의 채널을 통해 사용함으로써 쉽게 오차를 보정할 수 있다.


<br>from 위키피디아 https://ko.wikipedia.org/wiki/GPS#%EC%9C%84%EC%B9%98_%EA%B3%84%EC%82%B0

## 사용자 데이터 정의

In [5]:
time_interval = 10   # 10s -> 1 value

path_dir ='./Result'
file_list = os.listdir(path_dir)


distance_interval = 50  #미터 단위


height = 173.8 #20대 남자 키 평균
weight = 74.8  #20대 남자 몸무게 평균
age = 23    #

run_MET = 8  #2m/s 정도로 달리면 이정도라고 합니다!

man_RMR = 13.7516 * height + 5.0033 * weight + 6.7550 * age + 66.4730   #Harris - Benedict 공식에 의한 남자 기초 대사령 공식

## 기초 대사량 공식 관련 사이트

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=kimpulse&logNo=220104158403

In [6]:
for idx, f_name in enumerate(file_list):
    data = pd.read_csv(osp.join(path_dir, f_name) , sep=',', header=None)
    df = pd.DataFrame(data)
    df = df.iloc[1:,:]
    df.columns = ['Index', 'ID', 'Date-Time', 'Flag', 'Latitude', 'Longitude']

    Latitude_list = []
    Longitude_list = []
    Time_list = []
    
    for i in range(math.floor(len(df)/time_interval)):
    
        #10개 추출한 뒤, 위도&경도 str to float 
        df_sample = df.iloc[i*time_interval : (i+1)*time_interval]
        df_sample = df_sample.astype({'Latitude': 'float'})
        df_sample = df_sample.astype({'Longitude': 'float'})
        
        #추출한 값들의 평균 append (이렇게 하면 자동으로 null값 빼고 평균 계산)
        Latitude_temp = df_sample['Latitude'].mean() / 100  #원래 100배 더 곱해져 있어서
        Longitude_temp = df_sample['Longitude'].mean() / 100 
        
        # list 저장
        Latitude_list.append(round(Latitude_temp , 6)) # 소수점 6자리 (0이 너무 많이 나와서 이동을 안한것처럼 인식해 바꿨습니다!)
        Longitude_list.append(round(Longitude_temp , 6)) # 소수점 6자리 (0이 너무 많이 나와서 이동을 안한것처럼 인식해 바꿨습니다!)
        Time_list.append(df_sample.iloc[-1].loc['Date-Time'])
        '''
        if(df_sample['Latitude'].isnull().sum() != 0):
            print(df_sample)
        '''
    # Distance 계산
    distance_list = []

    for j in range(len(Latitude_list) -1):
        location1 = (Latitude_list[j], Longitude_list[j] )
        location2 = (Latitude_list[j+1], Longitude_list[j+1] )
    

        distance = haversine(location1, location2, unit = 'm')
        distance_list.append(distance)

    distance_total = round(sum(distance_list),3)  #깔끔하게 보이려고 임의로 세자리까지 출력했습니다!
    
    print(distance_total,"미터")

    
    #필요한 데이터 추출
    df = pd.DataFrame(list(zip( Time_list[:-1],distance_list)),
                      columns= ['Date-time', 'distance'] )
    
    # 누적 Distance 계산
    df['cum_distance'] = df['distance'].cumsum()  #누적합 계산
    df['distance_flag'] = df['cum_distance'].apply(lambda x: int(x / distance_interval)) #인터벌 마다 바뀌는 것을 체크하는 컬럼!
    distance_flag_index = sorted( df['distance_flag'].value_counts().keys())  # flag 컬럼 유니크 값 추출
    
    
    # 0~인터벌 미터 구간 속력 계산
    df_final = df[df['distance_flag'].isin([0])]

    start_time_str = df_final.iloc[0].loc['Date-time']
    end_time_str = df_final.iloc[-1].loc['Date-time']
    
    start_time = datetime.datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f')  #이동거리의 셋째 자리가 처음으로 바뀐 시간
    end_time = datetime.datetime.strptime(end_time_str, '%Y-%m-%d %H:%M:%S.%f')      #이동거리의 셋째 자리가 마지막으로 같은 시간

    Distance_temp = df_final.iloc[-1].loc['cum_distance'] - 0                   #처음 시작 지점은 출발점이기 때문에
    Time_temp = (end_time -  start_time).total_seconds()                         # 시간차이 초 단위로 환산

    Speed_temp = Distance_temp / Time_temp   #거리 = 시속 * 속력

    df_final['speed'] = Speed_temp
    
    for i in distance_flag_index[1:]:  #0~인터벌 미터 구간 제외한 나머지 부분 추가로 더해주는 for문 (위의 속력 계산 부분과 로직은 동일!)
        df_temp = df[df['distance_flag'].isin([i])]

        start_time_str = df_temp.iloc[0].loc['Date-time']
        end_time_str = df_temp.iloc[-1].loc['Date-time']

        start_time = datetime.datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f')  #이동거리의 셋째 자리가 처음으로 바뀐 시간
        end_time = datetime.datetime.strptime(end_time_str, '%Y-%m-%d %H:%M:%S.%f')      #이동거리의 셋째 자리가 마지막으로 같은 시간

        Distance_temp = df_temp.iloc[-1].loc['cum_distance'] - df_temp.iloc[0].loc['cum_distance'] #맨 마지막 누적 거리 - 맨 처음 누적거리
        Time_temp = (end_time -  start_time).total_seconds()                              # 시간차이 초 단위로 환산

        Speed_temp = Distance_temp / Time_temp

        df_temp['speed'] = Speed_temp  #100m 단위로 끊은 그 구간에서의 속력

        df_final = pd.concat([df_final,df_temp])
        
    #칼로리 계산
    df_final['kcal'] = df_final['speed'].apply(lambda x: (run_MET * (3.5 * weight * x  / distance_interval) /1000 ) * 5)
    
    total_kcal = round(sum(df_final['kcal'].unique()) ,3)   #깔끔하게 보이려고 임의로 세자리까지 출력했습니다!
    print(total_kcal, 'kcal을 소모했습니다')
    
    #하루 활동량 판단
    if(total_kcal < 1.2 * man_RMR):
        print('거의 앉아서 일함')
        
    elif(total_kcal >= 1.2 * man_RMR & total_kcal < 1.375 * man_RMR):
        print('적은 활동량')
        
    elif(total_kcal >= 1.375 * man_RMR & total_kcal < 1.55 * man_RMR):
        print('평균적인 활동량')
    
    elif(total_kcal >= 1.55 * man_RMR & total_kcal < 1.725 * man_RMR):
        print('많은 활동량')
        
    elif(total_kcal >= 1.725 * man_RMR & total_kcal < 1.9 * man_RMR):
        print('아주 많은 활동량')
        
    else:
        print('극한의 활동량')
        
    print()
    

    # 데이터 저장
    df_final.to_csv(osp.join('Result', 'processed_data_{}.csv'.format(idx)))

    


383.862 미터
1.243 kcal을 소모했습니다
거의 앉아서 일함



<ipython-input-6-c9d0d53bf908>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['speed'] = Speed_temp
<ipython-input-6-c9d0d53bf908>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['speed'] = Speed_temp  #100m 단위로 끊은 그 구간에서의 속력


395.152 미터
1.355 kcal을 소모했습니다
거의 앉아서 일함

229.718 미터
0.889 kcal을 소모했습니다
거의 앉아서 일함

251.635 미터
1.088 kcal을 소모했습니다
거의 앉아서 일함

16.089 미터
0.077 kcal을 소모했습니다
거의 앉아서 일함

330.023 미터
1.355 kcal을 소모했습니다
거의 앉아서 일함

273.911 미터
0.962 kcal을 소모했습니다
거의 앉아서 일함

192.433 미터
0.601 kcal을 소모했습니다
거의 앉아서 일함



In [7]:
df_final

,Date-time,distance,cum_distance,distance_flag,speed,kcal
0,2021-07-02 10:46:16.80,0.089697,0.089697,0,0.466889,0.097785
1,2021-07-02 10:46:17.80,0.000000,0.089697,0,0.466889,0.097785
2,2021-07-02 10:46:18.80,0.142863,0.232560,0,0.466889,0.097785
3,2021-07-02 10:46:19.80,0.111195,0.343755,0,0.466889,0.097785
4,2021-07-02 10:46:20.80,0.111195,0.454950,0,0.466889,0.097785
...,...,...,...,...,...,...
283,2021-07-02 10:50:59.80,0.894071,189.020533,3,0.879239,0.184148
284,2021-07-02 10:51:00.80,0.798771,189.819304,3,0.879239,0.184148
285,2021-07-02 10:51:01.80,0.929369,190.748674,3,0.879239,0.184148
286,2021-07-02 10:51:02.80,1.016707,191.765381,3,0.879239,0.184148
